Failed attempt with RBF SVM.

In [1]:
import pandas as pd
import numpy as np
from time import time
from sklearn.model_selection import train_test_split
# https://scikit-learn.org/stable/auto_examples/classification/plot_classifier_comparison.html
from sklearn.svm import SVC
from sklearn.metrics import precision_score, recall_score, f1_score
from tqdm import tqdm
import matplotlib.pyplot as plt

In [2]:
# https://www.kaggle.com/datasets/mlg-ulb/creditcardfraud
df = pd.read_csv('creditcard.csv')
print(df.columns)
print(df['Class'].value_counts())

Index(['Time', 'V1', 'V2', 'V3', 'V4', 'V5', 'V6', 'V7', 'V8', 'V9', 'V10',
       'V11', 'V12', 'V13', 'V14', 'V15', 'V16', 'V17', 'V18', 'V19', 'V20',
       'V21', 'V22', 'V23', 'V24', 'V25', 'V26', 'V27', 'V28', 'Amount',
       'Class'],
      dtype='object')
Class
0    284315
1       492
Name: count, dtype: int64


In [5]:
X_all = df[df.columns[:-1]].values
y_all = df['Class'].values

In [7]:
def fit_with_size(X_sample, y_sample, seed, train_size, verbose=False):
    X_train, X_test, y_train, y_test = train_test_split(
        X_sample, y_sample,
        train_size=train_size,
        stratify=y_sample,
        random_state=seed  # make less random
    )

    if verbose:
        print("Fitting model with", train_size, "of the data...", end=' ')
        start_time = time()
    np.random.seed(seed)  # make less random
    model = SVC(gamma=2, C=1, random_state=seed).fit(X_train, y_train)
    if verbose:
        end_time = time()
        print("done in", round(end_time-start_time), "seconds.")

    return model, X_test, y_test

In [9]:
def get_metrics(model, X_test, y_test):
    y_pred = []
    
    n_chunks = 100
    for i in tqdm(range(n_chunks)):
        chunk = X_test[i*n_chunks:(i+1)*n_chunks]
        chunk_pred = model.predict(chunk)
        y_pred.extend(chunk_pred)

    precision = precision_score(y_test, y_pred)
    recall = recall_score(y_test, y_pred)
    f1 = f1_score(y_test, y_pred)
    return precision, recall, f1

In [11]:
model, X_test, y_test = fit_with_size(
    X_all, y_all,
    seed=0, train_size=0.2,
    verbose=True
)

Fitting model with 0.2 of the data... done in 906 seconds.


In [12]:
get_metrics(model, X_test[:10000], y_test[:10000])

100%|████████████████████████████████████████████████████████████████████████████████| 100/100 [01:50<00:00,  1.10s/it]
C:\Users\ethan\AppData\Roaming\Python\Python311\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


(0.0, 0.0, 0.0)

In [13]:
np.where(y_test == 1)[0][:20]

array([  451,   629,  2019,  2621,  3149,  5289,  5376,  5711,  6001,
        7071,  7649,  7708,  7764,  7957,  9200,  9425, 10602, 10637,
       10846, 11697], dtype=int64)

In [14]:
# there's 2 positive samples in the first 1000 rows, but
# the model doesn't label them that way
p = model.predict(X_test[:1000])
sum(p)  # all 0's

0